In [21]:
import pandas as pd
import nltk

nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\giuli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\giuli\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [22]:
categories = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'hope']
categories_dfs = {}
for category in categories:
	categories_dfs[category] = pd.read_csv(f"./lexicon/{category}-NRC-Emotion-Lexicon.txt", header=None, names=['word', category], sep=None)

C:\Users\giuli\AppData\Local\Temp\ipykernel_38904\445186173.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  categories_dfs[category] = pd.read_csv(f"./lexicon/{category}-NRC-Emotion-Lexicon.txt", header=None, names=['word', category], sep=None)
C:\Users\giuli\AppData\Local\Temp\ipykernel_38904\445186173.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  categories_dfs[category] = pd.read_csv(f"./lexicon/{category}-NRC-Emotion-Lexicon.txt", header=None, names=['word', category], sep=None)
C:\Users\giuli\AppData\Local\Temp\ipykernel_38904\445186173.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this wa

In [23]:
lexicon = pd.DataFrame(columns=['word'])
for category in categories:
    lexicon = pd.merge(lexicon, categories_dfs[category], on='word', how='outer')
lexicon.dropna(inplace=True) # row with empty string
lexicon.sort_values(by='word', inplace=True)
lexicon

,word,anger,disgust,fear,joy,sadness,surprise,hope
11384,aback,0,0,0,0,0,0,0
11830,abacus,0,0,0,0,0,0,0
1324,abandon,0,0,1,0,1,0,0
663,abandoned,1,0,1,0,1,0,0
609,abandonment,1,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...
5281,zone,0,0,0,0,0,0,0
5458,zoo,0,0,0,0,0,0,0
7907,zoological,0,0,0,0,0,0,0
13967,zoology,0,0,0,0,0,0,0


In [24]:
#TRAIN_DATA_PATH = "./datasets/WASSA23_essay_level_with_labels_train.tsv"
#TRAIN_DATA_PATH = "./datasets/WASSA23_essay_level_dev_preproc.tsv"
TRAIN_DATA_PATH = "datasets/WASSA23_essay_level_test_preproc.tsv"
train_df = pd.read_csv(TRAIN_DATA_PATH, sep='\t')

In [25]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

preproc_lexicon = {}
lexicon['preproc_word'] = ''
for index, row in lexicon.iterrows():
    lemma = lemmatizer.lemmatize(row['word'])
    stemma = stemmer.stem(lemma)
    lexicon.loc[index, 'preproc_word'] = stemma
    if stemma in preproc_lexicon:
        preproc_lexicon[stemma] += lexicon.loc[index, categories] # |=
    else:
        preproc_lexicon[stemma] = lexicon.loc[index, categories]
preproc_lexicon = pd.DataFrame(preproc_lexicon).T
preproc_lexicon[preproc_lexicon > 0] = 1
preproc_lexicon = preproc_lexicon.loc[(preproc_lexicon!=0).any(axis=1)]
preproc_lexicon.to_csv('./lexicon/preproc_lexicon.csv')
lexicon.to_csv('./lexicon/lexicon.csv', index=False)
preproc_lexicon

,anger,disgust,fear,joy,sadness,surprise,hope
abandon,1,0,1,0,1,1,0
abduct,0,0,1,0,1,1,0
aberr,0,1,0,0,0,0,0
abhor,1,1,1,0,0,0,0
abhorr,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...
youth,1,0,1,1,0,1,0
zani,0,0,0,0,0,1,0
zeal,0,0,0,1,0,1,1
zealou,0,0,0,1,0,0,0


In [26]:
import re # python library to recoginze punctuation and digit
import contractions # python library to recoginze contractions

def remove_punctuations(text):
    return re.sub(r'[^\w\s]', '', text)

def remove_digit(text):
    return re.sub('\d+', '', text)

In [27]:
for category in categories:
	train_df[f'{category}_count'] = [0 for _ in range(len(train_df))]

for index, row in train_df.iterrows(): #TODO: controllare ( on no?)
	for word in row['essay']:
		if word in lexicon['word']:
			for category in categories:
				train_df.loc[index, f'{category}_count'] += lexicon.loc[word][category]

	essay = remove_punctuations(row['essay'])
	essay = remove_digit(essay)
	contracs = [contractions.fix(word) for word in essay.split()]
	lemmas = [lemmatizer.lemmatize(contraction) for contraction in contracs]
	stemmas = [stemmer.stem(lemma) for lemma in lemmas]

	for stemma in stemmas:
		if stemma in preproc_lexicon.index:
			for category in categories:
				train_df.loc[index, f'{category}_count'] += preproc_lexicon.loc[stemma][category]
train_df


,conversation_id,article_id,essay,speaker_id,gender,education,race,age,income,speaker_number,split,essay_id,anger_count,disgust_count,fear_count,joy_count,sadness_count,surprise_count,hope_count
0,1192,6,"hi my dear friend, how r you?. Are you used in...",45,2,6,1,25,50000,2,test,0,2,1,2,4,1,0,2
1,1192,6,The loss of human life is always a tragic even...,65,1,6,1,34,30000,1,test,1,9,3,10,3,11,2,2
2,1059,6,"Hello mate, it's been a long time since I met ...",53,2,6,1,64,40000,2,test,2,9,0,11,4,11,3,2
3,1059,6,us America and the security council adopts res...,92,2,6,1,35,5000,1,test,3,11,3,9,2,4,3,1
4,1060,6,My thought was NATO and government forces are ...,47,2,7,1,34,75000,1,test,4,10,3,14,0,10,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1183,19,Check out this story. It is an outrage that so...,143,2,6,1,42,50000,1,test,95,6,5,5,8,5,1,0
96,1184,19,I just read a story about a person who was bro...,145,1,4,1,63,55000,2,test,96,8,3,7,3,10,2,0
97,1184,19,I just read this article and it's really messe...,144,2,6,3,31,36000,1,test,97,9,6,9,2,9,1,0
98,1190,19,I just read about a man who is being deported ...,130,1,2,1,30,36000,2,test,98,5,3,5,3,6,1,1


In [28]:
for index, row in train_df.iterrows():
    word_count = len(row['essay'].split())
    for category in categories:
	    train_df.loc[index, f'{category}_count'] /= (word_count/100)

train_df
    

,conversation_id,article_id,essay,speaker_id,gender,education,race,age,income,speaker_number,split,essay_id,anger_count,disgust_count,fear_count,joy_count,sadness_count,surprise_count,hope_count
0,1192,6,"hi my dear friend, how r you?. Are you used in...",45,2,6,1,25,50000,2,test,0,3.448276,1.724138,3.448276,6.896552,1.724138,0.000000,3.448276
1,1192,6,The loss of human life is always a tragic even...,65,1,6,1,34,30000,1,test,1,6.338028,2.112676,7.042254,2.112676,7.746479,1.408451,1.408451
2,1059,6,"Hello mate, it's been a long time since I met ...",53,2,6,1,64,40000,2,test,2,7.258065,0.000000,8.870968,3.225806,8.870968,2.419355,1.612903
3,1059,6,us America and the security council adopts res...,92,2,6,1,35,5000,1,test,3,8.593750,2.343750,7.031250,1.562500,3.125000,2.343750,0.781250
4,1060,6,My thought was NATO and government forces are ...,47,2,7,1,34,75000,1,test,4,7.633588,2.290076,10.687023,0.000000,7.633588,2.290076,1.526718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1183,19,Check out this story. It is an outrage that so...,143,2,6,1,42,50000,1,test,95,4.225352,3.521127,3.521127,5.633803,3.521127,0.704225,0.000000
96,1184,19,I just read a story about a person who was bro...,145,1,4,1,63,55000,2,test,96,5.194805,1.948052,4.545455,1.948052,6.493506,1.298701,0.000000
97,1184,19,I just read this article and it's really messe...,144,2,6,3,31,36000,1,test,97,7.500000,5.000000,7.500000,1.666667,7.500000,0.833333,0.000000
98,1190,19,I just read about a man who is being deported ...,130,1,2,1,30,36000,2,test,98,4.545455,2.727273,4.545455,2.727273,5.454545,0.909091,0.909091


In [29]:
train_df.columns

Index(['conversation_id', 'article_id', 'essay', 'speaker_id', 'gender',
       'education', 'race', 'age', 'income', 'speaker_number', 'split',
       'essay_id', 'anger_count', 'disgust_count', 'fear_count', 'joy_count',
       'sadness_count', 'surprise_count', 'hope_count'],
      dtype='object')

In [30]:
#train_df.to_csv("datasets/train_essay_level_preproc.tsv", index=False, sep='\t')
#train_df.to_csv("datasets/dev_essay_level_preproc.tsv", index=False, sep='\t')
train_df.to_csv("datasets/test_essay_level_preproc.tsv", index=False, sep='\t')